In [84]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from time import time
import pyspark
import pyspark.sql.functions as F

In [85]:
# get the song list
song_lst = spark.read.csv('../../../../../../data/play/play_song.csv', header=True).cache()

In [110]:
song_lst.show(5)

+--------+---------+------------------+-----------+
| song_id|song_type|         song_name|     singer|
+--------+---------+------------------+-----------+
|  298250|        0|     See You Again|Miley Cyrus|
|21762903|        0|   电台情歌 (Live)|     莫文蔚|
| 6661513|        0|            干物女|       囧菌|
|21762903|        0|   电台情歌 (Live)|     莫文蔚|
|16889394|        0|一人饮酒醉唯美翻唱|     沈雨萱|
+--------+---------+------------------+-----------+
only showing top 5 rows



In [2]:
cd ~/DataScience/github

/Users/zirongwu/DataScience/github


In [3]:
ls

4_feature_label_generation_with_spark-Copy1.ipynb
5_train_model_sklearn.ipynb
CapstoneProjectCode.py
README.md
Yelp_Dataset_-_Data_Preprocessing.ipynb
Yelp_Dataset_-_NLP.ipynb
df_play_rc.csv
df_play_rc_fillna.csv
df_play_rc_nt.csv


In [4]:
df_play = pd.read_csv('df_play_rc.csv')

/Users/zirongwu/Documents/py3/py3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_play_nt = pd.read_csv('df_play_rc_nt.csv')

In [5]:
df_play_nt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169423 entries, 0 to 169422
Data columns (total 3 columns):
uid                   169423 non-null int64
song_id               169423 non-null int64
count_song_over_90    169423 non-null int64
dtypes: int64(3)
memory usage: 3.9 MB


In [5]:
df_play.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317171 entries, 0 to 317170
Data columns (total 3 columns):
uid                   317171 non-null int64
song_id               317171 non-null object
count_song_over_80    317171 non-null int64
dtypes: int64(2), object(1)
memory usage: 7.3+ MB


In [7]:
# remap the uid and song id
def new_ids(df, user_id, item_id):
    # build up the user id dictionary and item id dictionary
    user_id_list = df[user_id].unique()
    item_id_list = df[item_id].unique()
    user_id_dict = {v: k for k, v in enumerate(user_id_list)}
    item_id_dict = {v: k for k, v in enumerate(item_id_list)}
    df['uid_map'] = df.apply(lambda x: user_id_dict[x[user_id]], axis = 1)
    df['itemid_map'] = df.apply(lambda x: item_id_dict[x[item_id]], axis = 1)
    return df, user_id_dict, item_id_dict

In [8]:
# get the sparse matrix
def get_sparse_matrix(df, userid, item, value_col):
    highest_user_id = df[userid].max()
    highest_item_id = df[item].max()
    df_as_matrix = sparse.lil_matrix((highest_user_id, highest_item_id))
    for _, row in df.iterrows():
        df_as_matrix[row[userid] - 1, row[item] - 1] = row[value_col]
    return df_as_matrix

In [9]:
df_utility, uid_dict, songid_dict = \
                            new_ids(df_play_nt, user_id='uid', item_id='song_id')

In [10]:
len(uid_dict), len(songid_dict)

(19921, 52482)

In [11]:
df_utility.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169423 entries, 0 to 169422
Data columns (total 5 columns):
uid                   169423 non-null int64
song_id               169423 non-null int64
count_song_over_90    169423 non-null int64
uid_map               169423 non-null int64
itemid_map            169423 non-null int64
dtypes: int64(5)
memory usage: 6.5 MB


In [58]:
df_utility.head()

,uid,song_id,count_song_over_90,uid_map,itemid_map
0,168554187,6610601,6,0,0
1,168551752,74423,8,1,1
2,26841661,20273752,28,2,2
3,168546723,15216419,5,3,3
4,161870838,3214833,10,4,4


In [70]:
# Popularity Recommender
# recommend n most popular songs
n = 10
df_pop = df_utility.groupby('song_id').agg({'count_song_over_90': sum})

top_n_pop = df_pop.sort_values('count_song_over_90', ascending=False)[:10].index.tolist()

In [88]:
song_lst.where(F.col('song_id').isin(top_n_pop)).select(['song_name', 'singer']).show(5)

+---------------------------------+--------------+
|                        song_name|        singer|
+---------------------------------+--------------+
|                   一人有一个梦想|        黎瑞恩|
| 铃声多多_领音车载 Dj音少-够叼...|      未知歌手|
|铃声多多_六月枫宇辰寐加岛 - 不...|      未知歌手|
|                           逍遥叹|          胡歌|
|                       Dark Horse|Katy Perry&Tee|
+---------------------------------+--------------+
only showing top 5 rows



In [12]:
count_mat = get_sparse_matrix(df_utility, userid='uid_map', item='itemid_map', \
                              value_col='count_song_over_90')

In [13]:
count_mat

<19920x52481 sparse matrix of type '<class 'numpy.float64'>'
	with 169422 stored elements in LInked List format>

# Item-Item Simialrity recommender

In [14]:
# Item-Item similarity matrix
item_sim_mat = cosine_similarity(count_mat.T)

In [15]:
least_to_most_sim_indexes = np.argsort(item_sim_mat, axis = 1)

# set up the neighborhoods
neighborhood_size = 10
neighborhoods = least_to_most_sim_indexes[:, -neighborhood_size:]

In [16]:
# make a prediction for one user
user_id = 408

In [18]:
n_users = count_mat.shape[0]
n_items = count_mat.shape[1]

start_time = time()
items_rated_by_this_user = count_mat[user_id].nonzero()[1]

In [19]:
items_rated_by_this_user

array([  362,  2172,  5030,  7472, 21796, 28947, 29445, 34777, 36124,
       36571, 37004, 41885, 41993, 42497, 45637, 48046, 48391, 48746],
      dtype=int32)

In [20]:
out = np.zeros(n_items)
for item_to_rate in range(n_items):
    relevant_items = np.intersect1d(neighborhoods[item_to_rate],
                                   items_rated_by_this_user,
                                   assume_unique=True)
    out[item_to_rate] = count_mat[user_id, relevant_items] * \
        item_sim_mat[item_to_rate, relevant_items] / \
        item_sim_mat[item_to_rate, relevant_items].sum()

/Users/zirongwu/Documents/py3/py3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


In [21]:
pred_count = np.nan_to_num(out)
print(pred_count)

[0. 0. 0. ... 0. 0. 0.]


In [27]:
# get final recommendation
# recommend n songs
n = 5

# get the item indexes sorted by predicted count
item_index_sorted_by_pred_count = list(np.argsort(pred_count))[::-1]

# find items that have been rated by user
items_rated_by_this_user = count_mat[user_id].nonzero()[1]

In [30]:
# exclude the items that have been played by user
uncount_items_by_pred_play = [item for item in item_index_sorted_by_pred_count \
                             if item not in items_rated_by_this_user]
uncount_items_by_pred_play[:n]

[743, 7519, 13105, 36596, 30291]

In [115]:
song_id_top_n = uncount_items_by_pred_play[:n]
reco_song_lst = []
for i in song_id_top_n:
    reco_song_lst.append(df_utility[df_utility.itemid_map == i]['song_id'].unique()[0])

In [120]:
# reset the list itme datatype
reco_song_lst = list(map(int, reco_song_lst))
print('song ids: %s' %(reco_song_lst))
song_lst.where(F.col('song_id').isin(reco_song_lst)).select(['song_id', 'song_name', 'singer']).distinct().show(20)


song ids: [10609428, 324243, 673427, 6654421, 23155909, 13764870, 22800816, 9999059, 165582, 3434055]
+--------+----------------------------------+---------------+
| song_id|                         song_name|         singer|
+--------+----------------------------------+---------------+
| 9999059|                       玫瑰 (Live)|         张婧懿|
| 6654421|就是喜欢你-(电影《泡上美女总裁3...|     萌萌哒天团|
| 3434055|                    It&apos;s Time|Imagine Dragons|
|  165582|                            ??????|      ?????????|
|  165582|                              花火|         梁咏琪|
|22800816|                              冰雨|         刘德华|
|  673427|              I Can't Stop My L...|       动漫原声|
|  324243|                      蒲公英的约定|         周杰伦|
|10609428|                        一千零一夜|         邰正宵|
|23155909|    5D唯美360°环绕-最美旋律超好...|       一人众天|
|13764870|                    思思念念全是你|      高安,沁思|
| 3434055|                         It's Time|Imagine Dragons|
+--------+----------------------------------+---

# Matrix Factorization recommender

In [33]:
from sklearn.decomposition import NMF

def fit_nmf(Matrix, n_comps):
    nmf = NMF(n_components=n_comps, random_state = 42)
    nmf.fit(Matrix)
    W = nmf.transform(Matrix)
    H = nmf.components_
    err = nmf.reconstruction_err_
    return W, H, err

In [37]:
W, H, error = fit_nmf(count_mat, 300)
print('Shape of W: %s, \n, Shape of H: %s, \n, Error: %s' %(W.shape, H.shape, error))

Shape of W: (19920, 300), 
, Shape of H: (300, 52481), 
, Error: 6103.1165007503205


In [40]:
count_mat.shape

(19920, 52481)

In [38]:
count_mat_nmf = W.dot(H)

errors = np.array((count_mat - count_mat_nmf).flatten()).squeeze()

mask = np.array((count_mat.todense()).flatten()).squeeze()>0

mse = np.mean(errors[mask]**2)
avg_abs_err = abs(errors[mask]).mean()

In [53]:
print('MSE: %s, avg abs errors: %s' %(mse, avg_abs_err))

MSE: 208.7202707959006, avg abs errors: 9.359638864917365


In [55]:
# get recommendations for one user
user_id = 408
n = 10

pred_count = count_mat_nmf[user_id, :]
item_index_sorted_by_pred_count = list(np.argsort(pred_count))[::-1]

items_count_by_this_user = count_mat[user_id].nonzero()[1]

unplayed_item_by_pred_count = [i for i in item_index_sorted_by_pred_count \
                              if i not in items_count_by_this_user]

unplayed_item_by_pred_count[:n]

[2014, 2300, 14771, 9740, 1702, 367, 4988, 803, 1612, 9702]

In [121]:
song_id_top_n_nmf = unplayed_item_by_pred_count[:n]
reco_song_lst_nmf = []
for i in song_id_top_n_nmf:
    reco_song_lst_nmf.append(df_utility[df_utility.itemid_map == i]['song_id'].unique()[0])

In [134]:
reco_song_lst_nmf = map(int, reco_song_lst_nmf)
userid = df_utility[df_utility.uid_map == user_id].uid.unique()[0]

# reset the list itme datatype
reco_song_lst_nmf = list(map(int, reco_song_lst_nmf))
print('Recommendations for userid: %s' %(userid))
#print('song ids: %s' %(reco_song_lst_nmf))
song_lst.where(F.col('song_id').isin(reco_song_lst_nmf)).select(['song_id', 'song_name', 'singer']).distinct().show(20)

Recommendations for userid: 167611991
+-------+---------------+------------------------+
|song_id|      song_name|                  singer|
+-------+---------------+------------------------+
| 363402|       擦肩而过|                  宇桐非|
| 317514|        My Girl|                 BIGBANG|
| 313033|       三万英尺|                迪克牛仔|
|3400250|???????????????|            ????????????|
|3400250|     伤爱的理由|                上官晓懿|
|3034115|     一亿个伤心|                  蒙面哥|
|9871699|      吉米,来吧|谭晶[尖耳朵的阿凡达妹妹]|
|6449273|         ??????|               ?????????|
|9871699|      吉米 来吧|谭晶[尖耳朵的阿凡达妹妹]|
|6449273|           绅士|                  薛之谦|
|1104408|       第一夫人|                    张杰|
|6529417|         萌萌哒|              萌萌哒天团|
|2280171|       专属天使|                    群星|
|6449273|           绅士|                    绅士|
+-------+---------------+------------------------+



# Pyspark ALS

In [135]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [141]:
df_als_pd = spark.createDataFrame(df_utility)

In [142]:
df_als_pd.printSchema()

root
 |-- uid: long (nullable = true)
 |-- song_id: long (nullable = true)
 |-- count_song_over_90: long (nullable = true)
 |-- uid_map: long (nullable = true)
 |-- itemid_map: long (nullable = true)



In [136]:
df_als = spark.read.csv('/Users/zirongwu/DataScience/github/df_play_rc_nt.csv', header=True).cache()

In [138]:
df_als = df_als.withColumn('play_count', F.col('count_song_over_90').cast('int'))

In [144]:
# split the train and test data
(train, test) = df_als_pd.randomSplit([0.8, 0.2], seed = 42)

In [146]:
# build the recommedation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol='uid', itemCol='song_id', ratingCol='count_song_over_90',\
          coldStartStrategy='drop')
model = als.fit(train)

In [147]:
# evaluate the model by computing the RMSE on the test data
pred = model.transform(test)
evaluator = RegressionEvaluator(metricName='rmse', labelCol='count_song_over_90', predictionCol='prediction')
rmse = evaluator.evaluate(pred)
print('RMSE = %s' %(rmse))

RMSE = 72.19790062513691


In [148]:
# generate top 10 recommendations
userrc = model.recommendForAllUsers(10)
songrc = model.recommendForAllItems(10)

Recommendation for top 10 users: DataFrame[uid: int, recommendations: array<struct<song_id:int,rating:float>>]
DataFrame[song_id: int, recommendations: array<struct<uid:int,rating:float>>]


In [150]:
userrc.show()

+---------+--------------------+
|      uid|     recommendations|
+---------+--------------------+
|167630570|[[4652672, 5276.9...|
|168001770|[[4652672, 640.63...|
|168325610|[[4652672, 227.05...|
|168682030|[[4319581, 114.26...|
|168691740|[[4652672, 290.95...|
|168768770|[[4319581, 307.26...|
|168872440|[[4319581, 1536.3...|
|167738411|[[4194111, 502.47...|
|167857011|[[4652672, 184.41...|
|168461051|[[4652672, 139.69...|
|168579111|[[4652672, 627.64...|
|168751041|[[4652672, 240.87...|
|168775641|[[4284080, 373.73...|
| 87772572|[[4652672, 1545.1...|
|167670172|[[4319581, 197.42...|
|167735352|[[4652672, 3256.7...|
|167788732|[[4652672, 391.75...|
|167793682|[[4652672, 1536.9...|
|167822962|[[4284080, 1924.5...|
|167870462|[[4410466, 251.94...|
+---------+--------------------+
only showing top 20 rows



In [151]:
songrc.show()

+-------+--------------------+
|song_id|     recommendations|
+-------+--------------------+
|  89385|[[168710826, 102....|
| 104656|[[168391863, 362....|
| 109172|[[167903213, 237....|
| 110081|[[168391863, 186....|
| 110904|[[168391863, 103....|
| 118989|[[168576842, 2136...|
| 130003|[[168275343, 175....|
| 133948|[[168239582, 704....|
| 156296|[[168391863, 45.0...|
| 156365|[[168508050, 558....|
| 197258|[[168485439, 435....|
| 200878|[[168485439, 505....|
| 223021|[[167903213, 137....|
| 235318|[[168485439, 1056...|
| 237810|[[168239582, 1073...|
| 255362|[[168391863, 125....|
| 291764|[[168239582, 273....|
| 294870|[[168485439, 429....|
| 298862|[[168064494, 284....|
| 324155|[[168576842, 326....|
+-------+--------------------+
only showing top 20 rows



In [168]:
userec = userrc.filter(F.col('uid') == 167630570).select(F.col('recommendations')).collect()[0].asDict('song_id')

In [170]:
type(userec)

dict